In [ ]:
import os

In [ ]:
dataset_records = {
    "Appenzeller-Herzog_2019": 2873,
    "Bos_2018": 4878,
    "Brouwer_2019": 38114,
    "Chou_2003": 1908,
    "Chou_2004": 1630,
    "Donners_2021": 258,
    "Hall_2012": 8793,
    "Jeyaraman_2020": 1175,
    "Leenaars_2019": 5812,
    "Leenaars_2020": 7216,
    "Meijboom_2021": 882,
    "Menon_2022": 975,
    "Moran_2021": 5214,
    "Muthu_2021": 2719,
    "Nelson_2002": 366,
    "Oud_2018": 952,
    "Radjenovic_2013": 5935,
    "Sep_2021": 271,
    "Smid_2020": 2627,
    "van_de": 4544,
    "Valk_2021": 725,
    "van_der": 1970,
    "van_Dis": 9128,
    "Walker_2018": 48375,
    "Wassenaar_2017": 7668,
    "Wolters_2018": 4280
}

In [ ]:
# dict keys to list 
files = list(dataset_records.keys())

In [ ]:
def generate_job(setting, files, model, fe):
  for dataset in files:
    template = f"""apiVersion: batch/v1
kind: Job
metadata:
  name: sim-{dataset.lower().replace("_","-")[:-4]}-{model.lower().replace('_','-')}-{fe.lower().replace('_','-')}
spec:
  ttlSecondsAfterFinished: 100
  template:
      metadata:
        labels:
          app: simulation
      spec:
        restartPolicy: Never
        containers:
        - name: sim
          image: ghcr.io/jteijema/asreview-simulation-project:latest
          resources:
            requests:
              cpu: "1"
          env:
          - name: DATASET
            value: {dataset[:-4]}
          - name: SETTINGS
            value: "{setting}"
          envFrom:
            - secretRef:
                name: exoscale-credentials"""
    # save template to file
    with open(f"jobs/sim-{model}-{fe}-{dataset.lower().replace('_','-')[:-4]}.yaml", "w") as f:
        f.write(template)
    

In [ ]:
# settings generator (no nb)
!mkdir jobs

settings = []

exclude = ["nn-2-layer:tfidf"]

for model in ["logistic", "rf", "svm"]:                                                    # "logistic", "nn-2-layer", "xgboost", "rf", "svm"
    for fe in ["tfidf", "doc2vec", "reuseable_MiniLM", "reuseable_sbert"]:                 # "tfidf", "doc2vec", ""reuseable_MiniLM", "reuseable_sbert"
                if not f"{model}:{fe}" in exclude:
                    setting = "-m {model} -e {fe}".format(model=model, fe=fe)
                    settings.append(setting)
                    generate_job(setting, files, model, fe)

In [ ]:
# get all jobs
jobs = os.listdir('jobs')
for job in jobs:
    #!kubectl create -f ./jobs/{job}
    print(job)